In [1]:
# Import the dependencies
import pandas as pd
import numpy as np
import requests

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the API key
from config import weather_api_key
# Import the datatime module from the datatime library
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations. 
lat = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lat, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
# # Create an empty list to hold the weather data
cities = []

for coordinate in coordinates:

    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    if city not in cities:
        cities.append(city)
len(cities)

715

In [4]:
# Create an empty list to hold the weather data
city_data = []
# Print the beginning of the logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

import time
import sys

# Create counters.
record_count = 1
set_count = 1

url = f"http://api.openweathermap.org/data/2.5/weather?&units=imperial&appid={weather_api_key}"

# Loop through all the cities in our list.
for i, city in enumerate(cities):

    time.sleep(1)
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city
    city_url = url + "&q=" + f"{city}"
    # Log the URL, record and set numbers and the city 
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count
    record_count += 1
    # Run an API request for each of the cities.
    try: 
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        # Parse out the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_discription = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                            "Lat": city_lat,
                            "Lng": city_lng,
                            "Max Temp": city_max_temp,
                            "Humidity": city_humidity,
                            "Cloudiness": city_clouds,
                            "Wind Speed": city_wind,
                            "Country": city_country,
                            "Weather Description": city_discription})

# If an error is experienced, skip the city.                           
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | hasaki
Processing Record 2 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 3 of Set 1 | cape town
Processing Record 4 of Set 1 | rio grande
Processing Record 5 of Set 1 | hilo
Processing Record 6 of Set 1 | ushuaia
Processing Record 7 of Set 1 | yar-sale
Processing Record 8 of Set 1 | barrow
Processing Record 9 of Set 1 | puerto ayora
Processing Record 10 of Set 1 | urusha
Processing Record 11 of Set 1 | punta arenas
Processing Record 12 of Set 1 | puri
Processing Record 13 of Set 1 | hobart
Processing Record 14 of Set 1 | voh
Processing Record 15 of Set 1 | wanning
Processing Record 16 of Set 1 | lata
Processing Record 17 of Set 1 | bengkulu
Processing Record 18 of Set 1 | yumen
Processing Record 19 of Set 1 | ribeira grande
Processing Record 20 of Set 1 | albany
Processing Record 21 of Set 1 | maceio
Processing Record 22 of Set 1 | nikolskoye
Processing Record 23 of Set 1 | l

In [5]:
weather_data_df = pd.DataFrame(city_data)
weather_data_df.head(10)

City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0        Hasaki  35.7333  140.8333     64.80        76          97   
1     Cape Town -33.9258   18.4232     56.12        90          20   
2    Rio Grande -32.0350  -52.0986     50.95        68          28   
3          Hilo  19.7297 -155.0900     91.27        69          90   
4       Ushuaia -54.8000  -68.3000     29.86       100          75   
5      Yar-Sale  66.8333   70.8333     39.34        72          22   
6        Barrow  71.2906 -156.7887     33.82        75          90   
7  Puerto Ayora  -0.7393  -90.3518     80.56        88          43   
8        Urusha  54.0500  122.9000     50.58        93         100   
9  Punta Arenas -53.1500  -70.9167     35.71        64           0   

   Wind Speed Country Weather Description  
0        7.54      JP     overcast clouds  
1        2.17      ZA          few clouds  
2        9.35      BR    scattered clouds  
3       13.80      US     overcast clouds  
4        4.61      AR          light snow  
5       12.21      RU          few clouds  
6       14.97      US     overcast clouds  
7        5.99      EC    scattered clouds  
8        9.19      RU     overcast clouds  
9        5.03      CL           clear sky

In [6]:
# Create the output file 
output_data_file = "weather_data/WeatherPy_Database.csv"

# Export the city data into a cSV
weather_data_df.to_csv(output_data_file, index_label="City_ID")